# Title
[]()

In [2]:
from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")
from silvhua import *
from datetime import datetime
# from plotly.subplots import make_subplots
# import requests
import json

In [3]:
from processing import *

In [4]:
with open("credentials.json") as f:
    credentials = json.load(f)

ig_user_id = credentials['am_ig_user_id']
access_token = credentials['am_ig_access_token']

# Load posts

In [5]:
json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw'
csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'
filename='AM_IG_posts_2022-01-01_2023-03-01.sav'
response_json_dict = loadpickle(filename, json_path)
data_processed, count_vector, vect = response_json_dict_to_df(response_json_dict)
data_processed.head()


Time completed: 2023-03-07 16:05:32.737712
Time zone: Canada/Pacific


C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src\processing.py:37: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['year-month'] = df[timestamp_column].dt.to_period('M').dt.start_time # first day of the month
C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src\processing.py:38: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['year-week'] = df[timestamp_column].dt.to_period('W').dt.start_time # First day of the week (same day of week as for Jan 1 of that year)


Token pattern: (?u)\b\w\w+\b
Shape of vector array:  (362, 2905)

Top 10 words:
<number>    597
want        230
get         222
time        221
look        175
body        167
train       161
make        157
week        153
work        152
dtype: int64
Time processed: 2023-03-07 16:05:36.876935


,timestamp,caption,like_count,comments_count,media_type,media_product_type,media_url,permalink,id,comments.data,...,date,year,month,week_of_year,day_of_week,day_of_week_name,time,hour,year-month,year-week
0,2023-02-28 11:08:07-08:00,🎯 nutrition hierarchy importance 🎯 note st...,7,1,IMAGE,FEED,https://scontent.cdninstagram.com/v/t51.29350-...,https://www.instagram.com/p/CpN51UjB9mf/,17968352051238783,"[{'timestamp': '2023-02-28T19:35:06+0000', 'te...",...,2023-02-28,2023,2,9,1,Tuesday,11:08:07,11,2023-02-01,2023-02-27
1,2023-02-27 10:54:58-08:00,self conscious almost ruin may sound dramatic...,59,0,VIDEO,REELS,https://video.cdninstagram.com/o1/v/t16/f1/m82...,https://www.instagram.com/reel/CpLTguzhkB5/,17984097730814170,NaN,...,2023-02-27,2023,2,9,0,Monday,10:54:58,10,2023-02-01,2023-02-27
2,2023-02-26 11:21:55-08:00,devani ✅ mother ✅ radiographer originally sign...,15,1,CAROUSEL_ALBUM,FEED,https://scontent.cdninstagram.com/v/t51.29350-...,https://www.instagram.com/p/CpIx0vrBXQW/,17997080695727863,"[{'timestamp': '2023-02-26T19:44:41+0000', 'te...",...,2023-02-26,2023,2,8,6,Sunday,11:21:55,11,2023-02-01,2023-02-20
3,2023-02-23 11:00:09-08:00,‘ alone little together achieve much ’ tag pe...,19,0,IMAGE,FEED,https://scontent.cdninstagram.com/v/t51.29350-...,https://www.instagram.com/p/CpBA8mDhJ7W/,17915392925692330,NaN,...,2023-02-23,2023,2,8,3,Thursday,11:00:09,11,2023-02-01,2023-02-20
4,2023-02-22 10:54:18-08:00,deirdre hit first chin day hit two want know...,33,7,VIDEO,REELS,NaN,https://www.instagram.com/reel/Co-bX3TL7b6/,17961721733161620,"[{'timestamp': '2023-02-22T21:48:03+0000', 'te...",...,2023-02-22,2023,2,8,2,Wednesday,10:54:18,10,2023-02-01,2023-02-20


In [ ]:
data = pd.concat([json_normalize(response['data']) for key, response in response_json_dict.items()])

In [52]:
data.index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9,
            ...
             2,  3,  4,  5,  6,  7,  8,  9, 10, 11],
           dtype='int64', length=362)

In [6]:
caption_column='caption'
data_processed.columns.str.replace(caption_column, f'post_{caption_column}')

Index(['timestamp', 'post_caption', 'like_count', 'comments_count',
       'media_type', 'media_product_type', 'media_url', 'permalink', 'id',
       'comments.data', 'thumbnail_url', 'comments.paging.cursors.after',
       'comments.paging.next', 'date', 'year', 'month', 'week_of_year',
       'day_of_week', 'day_of_week_name', 'time', 'hour', 'year-month',
       'year-week'],
      dtype='object')

# Version 1

## `plot_images_tfidf`

In [7]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
def plot_images_tfidf(input_df, count_vector,
        n=5, top=True, max_columns=5, streamlit=False, 
        caption_column='caption', timezone='Canada/Pacific'):
    """
    Plot the images/video thumbnails of either the top or 
    worst performing instagram media (posts, reels, carousels).
    The following data is shown for each resulting media item:
        - Image/video thumbnail
        - Time stamp of the post in the provided time zone or in UTC time.
        - Number of likes and number of comments.
        - 5 words with the highest Tf-idf scores.

    Parameters:
        - df: DataFrame with the processed data.
        - count_vector: DataFrame with the count vectors.
        - n (int): Number of images to show.
        - top (bool): If True, plot images with the highest number of likes in 
            descending order. If False, plot images with the highest number of likes in
            ascending order. sort_by = ['like_count', 'comments_count', 'timestamp']
        - streamlit (bool): Whether or not the app runs on Streamlit. If False, then
            it is run on local machine.
        - timezone (str): Timezone parameter for the `.astimezone()` method.
            e.g. 'Australia/Sydney', 'Canada/Pacific'
    Returns:
        - DataFrame containing the data of the posts in the figure.
        - fig: Plotly figure object.
    """
    tfidf = tfidf_transform(count_vector)
    ncols = n if n<max_columns else max_columns
    nrows = (n + ncols - 1) // ncols
    sort_by = ['like_count', 'comments_count', 'timestamp']
    # SH 2023-03-05 21:52
    df = input_df.copy()
    df.columns = df.columns.str.replace(caption_column, f'media_{caption_column}')
    df = pd.concat([df, tfidf], axis=1)

    posts = df.sort_values(by=sort_by, ascending=False if top else True).head(n).copy()
    posts['thumbnail_url'].fillna(posts['media_url'], inplace=True)
    if timezone:
        print('Time zone:', timezone)
        converted_timestamp = [timestamp.astimezone(timezone) for timestamp in pd.to_datetime(posts['timestamp'])]
        titles = tuple(timestamp.strftime('%Y-%m-%d at %H:%M') for timestamp in converted_timestamp)
    else:
        titles = tuple(posts['timestamp'].dt.strftime('%Y-%m-%d at %H:%M').values.tolist())
    fig = make_subplots(
        rows=nrows, cols=ncols, 
        vertical_spacing=0.55/nrows
        )
    for index, (i, n_likes, n_comments, url, title) in enumerate(zip(posts.index,
            posts['like_count'], posts['comments_count'], posts['thumbnail_url'],
            titles)
        ):
        highest_tfidf = [word for word, value in tfidf.loc[i].sort_values(
            ascending=False).head(5).items() if value > 0]
        fig.add_layout_image(
            x=0, y=0,
            xanchor='center', yanchor='middle',
            sizex=1, sizey=1,
            row=index // ncols + 1,
            col=index % ncols + 1,
            xref="x",
            yref="y",
            opacity=1.0,
            source=url
        )
        annotation_y_position = -0.075
        fig.add_annotation(
            xref="x domain",
            yref="y domain",
            x=0.5,
            y=1.1,
            text=title,
            axref="x domain",
            ayref="y",
            ax=0.5,
            ay=2,
            arrowhead=2,
            row=index // ncols + 1,
            col=index % ncols + 1,
        )
        fig.add_annotation(
            xref="x domain",
            yref="y domain",
            x=0.5,
            y=annotation_y_position,
            text=f'{n_likes} likes, {n_comments} comments',
            axref="x domain",
            ayref="y",
            ax=0.5,
            ay=2,
            arrowhead=2,
            row=index // ncols + 1,
            col=index % ncols + 1,
        )
        annotation_y_position -= 0.1
        fig.add_annotation(
            xref="x domain",
            yref="y domain",
            x=0.5,
            y=annotation_y_position,
            text=f'Most unique words:',
            axref="x domain",
            ayref="y",
            ax=0.5,
            ay=2,
            arrowhead=2,
            row=index // ncols + 1,
            col=index % ncols + 1,
        )
        for word in highest_tfidf:
            annotation_y_position -= .075
            fig.add_annotation(
                xref="x domain",
                yref="y domain",
                x=0.5,
                y=annotation_y_position,
                text=word,
                axref="x domain",
                ayref="y",
                ax=0.5,
                ay=2,
                arrowhead=2,
                row=index // ncols + 1,
                col=index % ncols + 1,
                # hovertext=caption,
            )
    fig.update_xaxes(range=[-0.5,0.5], showticklabels=False)
    fig.update_yaxes(range=[-0.5,0.5], showticklabels=False)
    fig.update_layout(
        title_text='Posts with highest number of likes' if top==True else 'Posts with fewest number of likes',
        title_xanchor='center', title_x=0.5,
        plot_bgcolor="white", 
        height = 120 + nrows * 300,
        margin_b=120, margin_l=50, margin_r=50,
        )
    if streamlit:
        st.plotly_chart(fig, use_container_width=True)
    else:
        fig.show()
    return posts.reset_index(drop=True), fig


top_posts, fig = plot_images_tfidf(data_processed, count_vector)

Time zone: Canada/Pacific


In [8]:
bottom_posts, fig_bottom = plot_images_tfidf(data_processed, count_vector, top=False)

Time zone: Canada/Pacific


In [9]:
bottom_posts.loc[0, 'media_url']

'https://video.cdninstagram.com/o1/v/t16/f1/m82/43474E1AD9BC19BAEE84FA4BA76079AA_video_dashinit.mp4?efg=eyJ2ZW5jb2RlX3RhZyI6InZ0c192b2RfdXJsZ2VuLjcyMC5jbGlwcyJ9&_nc_ht=video.cdninstagram.com&_nc_cat=103&vs=1557881478059002_3784848049&_nc_vs=HBksFQIYT2lnX3hwdl9yZWVsc19wZXJtYW5lbnRfcHJvZC80MzQ3NEUxQUQ5QkMxOUJBRUU4NEZBNEJBNzYwNzlBQV92aWRlb19kYXNoaW5pdC5tcDQVAALIAQAVABgkR0Yzc3BCTk1QNGVXNGhvREFIa3hFOGZyR1d0M2JxX0VBQUFGFQICyAEAKAAYABsBiAd1c2Vfb2lsATEVAAAm2u327Kr33j8VAigCQzMsF0BWfrhR64UfGBJkYXNoX2Jhc2VsaW5lXzFfdjERAHUAAA%3D%3D&ccb=9-4&oh=00_AfCpyRXEV-WYNr1NXsRuOcHS84s80pu2bIjEshizKq69wg&oe=6401E09C&_nc_sid=ea0b6e&_nc_rid=40b9bdac3e'

In [10]:
bottom_posts['media_caption'].values

array(['✅ lack energy complete day day task  ✅ affect family life  ✅ energy play kid  check devani ’ story ⬆️⬆️⬆️ #onlinecoach #testimonial #realresults #realpeople #fitness #nutrition #strongandsassyprogramme ',
       'expire tonight  invitation woman sick fad diet want see long term result    youre female want bikini confident year round <Number> week  last chance apply  id hate miss result like   aoibheann lose <Number> <Number> week  sarah bench old <Number> <Number> rep  trish lose inch kilo look <Number> kid  decision  super easy apply  hour leave apply  comment me  ill reach dm see youre good fit  ',
       'september strong sassy programme go better educational ever     know along nutrition accountability call every week  nutrition guideline  calorie calculate  also provide lot tasty recipe  im work finish touch app  incorporate train  nutrition accountability need  include <Number> tasty recipe  want first know drop dm sas   ',
       'nicole change mindset become confident <

In [11]:
top_posts['media_caption'].values

array(['best anniversary present ❤️  present feel little insignificant comparison 😂  <Handle> 💍 ',
       'happy <Number> year ❤️💕  maybe unfollow ’ sick sopiness  😂 ',
       '❤️❤️❤️ talk lot goal expectation  goal life like woman 🙏🙌 <Number> full life  absolute inspiration life 🥰 love heart ❤️ #grannysgirl ',
       '💁\u200d♀️   people say arnold schwarzenegger oh dont want look like   reply  dont worry  never     let tell  build muscle bloody hard  dont wake one day are bulky  😅😅😅   ive spend year improve get stronger  fitter healthier  goal  dont think lift afew weight youre go gain <Number> muscle look bulky   honestly  people never ever train hard train 😅   let also tell  majority photo take mid post train serious pump  normal clothe  im normal size <Number> <Number> female  dont jack quad shoulder vein 😅  even though id happy walk around like    moral story  people never train hard enough get bulky  people dont walk round jack 😅 toned  look want actually muscle  say want toned  

In [35]:
top_posts

,timestamp,media_caption,like_count,comments_count,media_type,media_product_type,media_url,permalink,id,comments.data,...,#weightlift,#weightloss,#weightlossjourney,#whendoesgymnasticsgeteasier,#whenfoodislife,#yachtclubsocial,#yourhealthisyourwealth,#yourhealthmatters,<number>,<numberzzznumber>
0,2022-08-18 13:04:45-07:00,best anniversary present ❤️ present feel litt...,1296,479,CAROUSEL_ALBUM,FEED,https://scontent.cdninstagram.com/v/t51.29350-...,https://www.instagram.com/p/ChaeGYxBaVQ/,17947293587166647,"[{'timestamp': '2023-02-03T04:13:14+0000', 'te...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,2022-08-20 01:41:07-07:00,happy <Number> year ❤️💕 maybe unfollow ’ sick...,307,2,IMAGE,FEED,https://scontent.cdninstagram.com/v/t51.29350-...,https://www.instagram.com/p/CheZdF4Bpy1/,17956591834959825,"[{'timestamp': '2022-09-17T11:35:32+0000', 'te...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.144044,0.0
2,2022-12-16 03:33:16-08:00,❤️❤️❤️ talk lot goal expectation goal life li...,284,36,VIDEO,REELS,https://video.cdninstagram.com/o1/v/t16/f1/m82...,https://www.instagram.com/reel/CmOivUnIr8-/,17978937280883442,"[{'timestamp': '2022-12-17T01:25:10+0000', 'te...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.090061,0.0
3,2022-01-26 23:05:12-08:00,💁‍♀️ people say arnold schwarzenegger oh don...,274,21,CAROUSEL_ALBUM,FEED,https://scontent.cdninstagram.com/v/t51.29350-...,https://www.instagram.com/p/CZOXhKohhDc/,17908881674262988,"[{'timestamp': '2022-01-27T20:26:29+0000', 'te...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.078046,0.0
4,2022-12-19 21:55:08-08:00,<Number> year later literally feel like yester...,272,28,VIDEO,REELS,NaN,https://www.instagram.com/reel/CmYPXlgorUy/,17959044758193298,"[{'timestamp': '2022-12-20T11:27:45+0000', 'te...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.097452,0.0


## `tfidf_top_vs_bottom`

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
def tfidf_top_vs_bottom(top_posts, bottom_posts, caption_column='media_caption'):
    top_posts_BoW = ''.join([word for doc in top_posts[caption_column] for word in doc])
    bottom_posts_BoW = ''.join([word for doc in bottom_posts[caption_column] for word in doc])
    docs = [top_posts_BoW, bottom_posts_BoW]
    
    count_vectorizer = CountVectorizer()
    count_vectors = count_vectorizer.fit_transform([doc for doc in docs])
    count_vectors = pd.DataFrame(
        count_vectors.toarray(), 
        columns=count_vectorizer.get_feature_names_out(),
        index=['top_posts', 'bottom_posts']
    )
    tfidf_vectorizer = TfidfVectorizer(smooth_idf=False)
    tfidf_vectors = tfidf_vectorizer.fit_transform([doc for doc in docs])
    tfidf_vectors = pd.DataFrame(
        tfidf_vectors.toarray(), columns=tfidf_vectorizer.get_feature_names_out(),
        index=['top_posts', 'bottom_posts'])
    print(f'Shape of tf-idf vector array: {tfidf_vectors.shape}')

    tfidf_vectors

    return tfidf_vectors, count_vectors

tfidf_vectors, count_vectors = tfidf_top_vs_bottom(top_posts, bottom_posts)
count_vectors

Shape of tf-idf vector array: (2, 261)


,absolute,accountability,achieve,actually,afew,affect,along,already,also,always,...,week,weight,wise,woman,work,worry,worth,year,yesterday,youre
top_posts,1,0,0,2,1,0,0,1,1,0,...,0,2,0,1,0,1,0,3,1,2
bottom_posts,0,2,1,0,0,1,1,0,1,1,...,8,0,1,2,4,0,1,3,0,4


In [27]:
count_vectors['nicole']

top_posts       0
bottom_posts    5
Name: nicole, dtype: int64

In [14]:
tfidf_vectors['anniversary']

top_posts       0.058458
bottom_posts    0.000000
Name: anniversary, dtype: float64

In [21]:
tfidf_vectors.loc['top_posts'][(tfidf_vectors.loc['top_posts'] > 0) & (tfidf_vectors.loc['bottom_posts'] == 0)]

absolute       0.058458
actually       0.116916
afew           0.058458
already        0.058458
anniversary    0.058458
                 ...   
walk           0.116916
wasnt          0.058458
weight         0.116916
worry          0.058458
yesterday      0.058458
Name: top_posts, Length: 94, dtype: float64

In [22]:
tfidf_vectors.loc['top_posts'][(tfidf_vectors.loc['top_posts'] > 0) & (tfidf_vectors.loc['bottom_posts'] == 0)].sort_values(ascending=False)

dont         0.409205
muscle       0.233832
people       0.233832
hard         0.175374
bulky        0.175374
               ...   
home         0.058458
heart        0.058458
healthier    0.058458
health       0.058458
yesterday    0.058458
Name: top_posts, Length: 94, dtype: float64

In [24]:
tfidf_vectors.loc['bottom_posts'][(tfidf_vectors.loc['top_posts'] == 0) & (tfidf_vectors.loc['bottom_posts'] > 0)].sort_values(ascending=False)

week           0.321108
apply          0.240831
result         0.240831
nicole         0.200692
confident      0.160554
                 ...   
educational    0.040138
drop           0.040138
doubt          0.040138
do             0.040138
worth          0.040138
Name: bottom_posts, Length: 142, dtype: float64

# Version 2

In [36]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
def plot_images_tfidf(input_df, count_vector,
        n=5, top=True, max_columns=5, streamlit=False, 
        caption_column='caption', timezone='Canada/Pacific'):
    """
    Plot the images/video thumbnails of either the top or 
    worst performing instagram media (posts, reels, carousels).
    The following data is shown for each resulting media item:
        - Image/video thumbnail
        - Time stamp of the post in the provided time zone or in UTC time.
        - Number of likes and number of comments.
        - 5 words with the highest Tf-idf scores.

    Parameters:
        - df: DataFrame with the processed data.
        - count_vector: DataFrame with the count vectors.
        - n (int): Number of images to show.
        - top (bool): If True, plot images with the highest number of likes in 
            descending order. If False, plot images with the highest number of likes in
            ascending order. sort_by = ['like_count', 'comments_count', 'timestamp']
        - streamlit (bool): Whether or not the app runs on Streamlit. If False, then
            it is run on local machine.
        - timezone (str): Timezone parameter for the `.astimezone()` method.
            e.g. 'Australia/Sydney', 'Canada/Pacific'
    Returns:
        - DataFrame containing the data of the posts in the figure.
        - fig: Plotly figure object.

    Updates:
    SH 2023-03-07 16:27: Maintain original index for subsequent indexing of dataframe with the raw data.
    """
    tfidf = tfidf_transform(count_vector)
    ncols = n if n<max_columns else max_columns
    nrows = (n + ncols - 1) // ncols
    sort_by = ['like_count', 'comments_count', 'timestamp']
    # SH 2023-03-05 21:52
    df = input_df.copy()
    df.columns = df.columns.str.replace(caption_column, f'media_{caption_column}')
    df = pd.concat([df, tfidf], axis=1)

    posts = df.sort_values(by=sort_by, ascending=False if top else True).head(n).copy()
    posts['thumbnail_url'].fillna(posts['media_url'], inplace=True)
    if timezone:
        print('Time zone:', timezone)
        converted_timestamp = [timestamp.astimezone(timezone) for timestamp in pd.to_datetime(posts['timestamp'])]
        titles = tuple(timestamp.strftime('%Y-%m-%d at %H:%M') for timestamp in converted_timestamp)
    else:
        titles = tuple(posts['timestamp'].dt.strftime('%Y-%m-%d at %H:%M').values.tolist())
    fig = make_subplots(
        rows=nrows, cols=ncols, 
        vertical_spacing=0.55/nrows
        )
    for index, (i, n_likes, n_comments, url, title) in enumerate(zip(posts.index,
            posts['like_count'], posts['comments_count'], posts['thumbnail_url'],
            titles)
        ):
        highest_tfidf = [word for word, value in tfidf.loc[i].sort_values(
            ascending=False).head(5).items() if value > 0]
        fig.add_layout_image(
            x=0, y=0,
            xanchor='center', yanchor='middle',
            sizex=1, sizey=1,
            row=index // ncols + 1,
            col=index % ncols + 1,
            xref="x",
            yref="y",
            opacity=1.0,
            source=url
        )
        annotation_y_position = -0.075
        fig.add_annotation(
            xref="x domain",
            yref="y domain",
            x=0.5,
            y=1.1,
            text=title,
            axref="x domain",
            ayref="y",
            ax=0.5,
            ay=2,
            arrowhead=2,
            row=index // ncols + 1,
            col=index % ncols + 1,
        )
        fig.add_annotation(
            xref="x domain",
            yref="y domain",
            x=0.5,
            y=annotation_y_position,
            text=f'{n_likes} likes, {n_comments} comments',
            axref="x domain",
            ayref="y",
            ax=0.5,
            ay=2,
            arrowhead=2,
            row=index // ncols + 1,
            col=index % ncols + 1,
        )
        annotation_y_position -= 0.1
        fig.add_annotation(
            xref="x domain",
            yref="y domain",
            x=0.5,
            y=annotation_y_position,
            text=f'Most unique words:',
            axref="x domain",
            ayref="y",
            ax=0.5,
            ay=2,
            arrowhead=2,
            row=index // ncols + 1,
            col=index % ncols + 1,
        )
        for word in highest_tfidf:
            annotation_y_position -= .075
            fig.add_annotation(
                xref="x domain",
                yref="y domain",
                x=0.5,
                y=annotation_y_position,
                text=word,
                axref="x domain",
                ayref="y",
                ax=0.5,
                ay=2,
                arrowhead=2,
                row=index // ncols + 1,
                col=index % ncols + 1,
                # hovertext=caption,
            )
    fig.update_xaxes(range=[-0.5,0.5], showticklabels=False)
    fig.update_yaxes(range=[-0.5,0.5], showticklabels=False)
    fig.update_layout(
        title_text='Posts with highest number of likes' if top==True else 'Posts with fewest number of likes',
        title_xanchor='center', title_x=0.5,
        plot_bgcolor="white", 
        height = 120 + nrows * 300,
        margin_b=120, margin_l=50, margin_r=50,
        )
    if streamlit:
        st.plotly_chart(fig, use_container_width=True)
    else:
        fig.show()
    return posts, fig


top_posts, fig = plot_images_tfidf(data_processed, count_vector)

Time zone: Canada/Pacific


In [38]:
top_posts

,timestamp,media_caption,like_count,comments_count,media_type,media_product_type,media_url,permalink,id,comments.data,...,#weightlift,#weightloss,#weightlossjourney,#whendoesgymnasticsgeteasier,#whenfoodislife,#yachtclubsocial,#yourhealthisyourwealth,#yourhealthmatters,<number>,<numberzzznumber>
166,2022-08-18 13:04:45-07:00,best anniversary present ❤️ present feel litt...,1296,479,CAROUSEL_ALBUM,FEED,https://scontent.cdninstagram.com/v/t51.29350-...,https://www.instagram.com/p/ChaeGYxBaVQ/,17947293587166647,"[{'timestamp': '2023-02-03T04:13:14+0000', 'te...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
165,2022-08-20 01:41:07-07:00,happy <Number> year ❤️💕 maybe unfollow ’ sick...,307,2,IMAGE,FEED,https://scontent.cdninstagram.com/v/t51.29350-...,https://www.instagram.com/p/CheZdF4Bpy1/,17956591834959825,"[{'timestamp': '2022-09-17T11:35:32+0000', 'te...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.144044,0.0
64,2022-12-16 03:33:16-08:00,❤️❤️❤️ talk lot goal expectation goal life li...,284,36,VIDEO,REELS,https://video.cdninstagram.com/o1/v/t16/f1/m82...,https://www.instagram.com/reel/CmOivUnIr8-/,17978937280883442,"[{'timestamp': '2022-12-17T01:25:10+0000', 'te...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.090061,0.0
344,2022-01-26 23:05:12-08:00,💁‍♀️ people say arnold schwarzenegger oh don...,274,21,CAROUSEL_ALBUM,FEED,https://scontent.cdninstagram.com/v/t51.29350-...,https://www.instagram.com/p/CZOXhKohhDc/,17908881674262988,"[{'timestamp': '2022-01-27T20:26:29+0000', 'te...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.078046,0.0
62,2022-12-19 21:55:08-08:00,<Number> year later literally feel like yester...,272,28,VIDEO,REELS,NaN,https://www.instagram.com/reel/CmYPXlgorUy/,17959044758193298,"[{'timestamp': '2022-12-20T11:27:45+0000', 'te...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.097452,0.0


In [40]:
bottom_posts, fig_bottom = plot_images_tfidf(data_processed, count_vector, top=False);

Time zone: Canada/Pacific


In [84]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
def tfidf_top_vs_bottom(data, top_posts, bottom_posts, caption_column='media_caption'):
    """
    data: DataFrame containing raw data from `get_user_ig_post_text` function
    """
    data = data.reset_index(drop=True)
    top_posts_index = top_posts.index
    bottom_posts_index = bottom_posts.index
    # top_posts_BoW = ''.join([word for doc in top_posts[caption_column] for word in doc])
    # bottom_posts_BoW = ''.join([word for doc in bottom_posts[caption_column] for word in doc])
    # docs = [top_posts_BoW, bottom_posts_BoW]
    select_posts = pd.concat([data.loc[top_posts_index], data.loc[bottom_posts_index]])
    posts_processed, count_vector, vect = post_preprocessing(select_posts)
    tfidf_vectorizer = TfidfVectorizer(smooth_idf=False)
    binary_count_vectorizer = (count_vector/count_vector).fillna(0).astype(int)
    top_posts_tfidf = tfidf_transform(binary_count_vectorizer)

    return binary_count_vectorizer, tfidf
    # return top_posts_count_vector/top_posts_count_vector, bottom_posts_count_vector

    # tfidf_vectorizer = TfidfVectorizer(smooth_idf=False)
    # tfidf_vectors = tfidf_vectorizer.fit_transform([doc for doc in docs])
    # tfidf_vectors = pd.DataFrame(
    #     tfidf_vectors.toarray(), columns=tfidf_vectorizer.get_feature_names_out(),
    #     index=['top_posts', 'bottom_posts'])
    # print(f'Shape of tf-idf vector array: {tfidf_vectors.shape}')
    # return tfidf_vectors, count_vectors, count_vectors_top

binary_count_vectorizer, tfidf = tfidf_top_vs_bottom(data, top_posts, bottom_posts)

Time zone: Canada/Pacific
Token pattern: (?u)\b\w\w+\b
Shape of vector array:  (10, 262)

Top 10 words:
<number>    20
want        10
like         8
week         8
dont         7
apply        6
year         6
youre        6
feel         6
result       6
dtype: int64
Time processed: 2023-03-07 17:14:30.289817


C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src\processing.py:37: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.

C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src\processing.py:38: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



In [85]:
binary_count_vectorizer

,absolute,accountability,achieve,actually,afew,affect,along,already,also,always,...,#onlinefitnesscoach,#realpeople,#realresults,#strength,#strengthtraining,#strongandsassyprogramme,#strongerfromtheinsideout,#strongwomen,#testimonial,<number>
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,1,1,0,0,1,1,0,...,1,0,0,1,1,0,1,1,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,0,0,0,0,0,1,0,0,0,0,...,0,1,1,0,0,1,0,0,1,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7,0,1,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
8,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [86]:
tfidf

,absolute,accountability,achieve,actually,afew,affect,along,already,also,always,...,#onlinefitnesscoach,#realpeople,#realresults,#strength,#strengthtraining,#strongandsassyprogramme,#strongerfromtheinsideout,#strongwomen,#testimonial,<number>
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.203445
2,0.298723,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.132607
3,0.000000,0.000000,0.000000,0.114078,0.114078,0.000000,0.000000,0.114078,0.096977,0.000000,...,0.114078,0.000000,0.000000,0.114078,0.114078,0.000000,0.114078,0.114078,0.000000,0.050641
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.130980
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.231118,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.231118,0.231118,0.000000,0.000000,0.231118,0.000000,0.000000,0.231118,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.077236
7,0.000000,0.173538,0.000000,0.000000,0.000000,0.000000,0.173538,0.000000,0.147524,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.077036
8,0.000000,0.000000,0.129899,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.129899,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.057664
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071980


# *End of Page*